# International status widget

Data model for the International Status widget:

{  
id [int],  
Location_id [str] location_id from location table  
indicator [str] one of (nationally_determined_contributions, forest_reference_emissions_levels, ipcc_wetlands_supplement)  
value [str] with the full phrase to display  
}  

Each of the phrases are constructed based on the logic explained.

In [1]:
import pandas as pd
import numpy as np


In [2]:
policy_data = pd.read_csv('../../../data/Climate_Policy_Data.csv')
column_names = {'NDC Target (mtCO2e/yr)':'ndc_target', 'NDC % Reduction Target':'pct_ndc_target',
       'NDC Target URL':'url', 'NDC Blurb':'blurb', 'Climate Vulnerability Rank (index)':'climate_vulnerability_rank',
       'NDC: first/updated':'ndc_updates', "NDC:includes/doesn't include":'ndc_inclusive',
       'NDC Mitigation (Y/N)':'ndc_mitigation', 'NDC Adaptation (Y/N)':'ndc_adaptation', 'FREL':'frel',
       'Forest or Wetland':'forest_wetland', 'Wetland Supplement':'wetland_suplement'}
policy_data.rename(columns=column_names, inplace=True)
policy_data['frel'] = np.float64(policy_data['frel'])
policy_data['forest_wetland'] = policy_data['forest_wetland'].str.lower()
policy_data.head()

,iso,name,pct_ndc_target,ndc_target,url,blurb,climate_vulnerability_rank,ndc_updates,ndc_inclusive,ndc_mitigation,ndc_adaptation,frel,forest_wetland,wetland_suplement
0,AGO,Angola,50.0,96.65,https://www4.unfccc.int/sites/NDCStaging/pages...,"""Angola plans to reduce GHG emissions up to 35...",106.67,Updated,Includes,No,Yes,NaN,NaN,Has not
1,ATG,Antigua & Barbuda,NaN,NaN,https://www4.unfccc.int/sites/NDCStaging/pages...,"Conditional Adaptation Targets "" (1) By 2025, ...",74.50,NaN,Doesn't include,No,No,NaN,NaN,Has not
2,AUS,Australia,28.0,161.00,https://www4.unfccc.int/sites/NDCStaging/pages...,"""Under a Paris Agreement applicable to all, Au...",52.00,Updated,Includes,Yes,No,NaN,not specified,Has
3,BHR,Bahrain,NaN,NaN,https://www4.unfccc.int/sites/NDCStaging/pages...,The Kingdom of Bahrain communicated in its NDC...,125.00,NaN,Doesn't include,No,No,NaN,NaN,Has not
4,BGD,Bangladesh,15.0,35.10,https://www4.unfccc.int/sites/NDCStaging/pages...,"""The NDC of Bangladesh consists of the followi...",25.00,NaN,Doesn't include,No,No,374253.0,NaN,Has not


## Nationally determined contributions (NDC)

For each country, example sentence:  
`Indonesia` NDC target is `1,181.21` mt CO₂e/yr by 2030.  
Country's `updated/first` NDC `includes/doesn’t include` coastal and marine NBS for `both mitigation and adaptation/mitigation only/adaptation only`  
  
Logic:
1. Country name and NDC target are provided in the data.  
2. If **ndc_updates** = *Updated* --> "updated". Else, "first"  
3. If **ndc_inclusive** = *Doesn't include* or empty, "Doesn't include...".
4. If **ndc_inclusive** = *Includes*, check combintion (yes/no) in **ndc_mitigation** and **ndc_adaptation**

In [3]:
#ndc = policy_data[['iso','name','ndc_target', 'ndc_updates','ndc_inclusive','ndc_mitigation','ndc_adaptation']].copy()
ndc = policy_data[['iso','name']].copy()
ndc.head()

,iso,name
0,AGO,Angola
1,ATG,Antigua & Barbuda
2,AUS,Australia
3,BHR,Bahrain
4,BGD,Bangladesh


In [4]:
no_commitment_phrase = 'has no NDC commitment'

ndc['NDC_target'] = np.where(policy_data['ndc_target'].isna(), no_commitment_phrase, policy_data['ndc_target'])
ndc['NDC_update'] = np.where(policy_data['ndc_target'].isna(), np.nan, np.where(policy_data['ndc_updates'] == 'Updated', 'updated NDC', 'first NDC'))

def ndc_incl(df):
    if((df['ndc_mitigation'] == 'Yes') & (df['ndc_mitigation'] == 'Yes')):
        return('includes coastal and marine NBS for both mitigation and adaptation')
    elif((df['ndc_mitigation'] == 'Yes') & (df['ndc_mitigation'] == 'No')):
        return('includes coastal and marine NBS for mitigation')
    elif((df['ndc_mitigation'] == 'No') & (df['ndc_mitigation'] == 'Yes')):
        return('includes coastal and marine NBS for adaptation')
    else:
        return('doesn\'t include coastal and marine NBS')

ndc['NDC_include'] = policy_data.apply(ndc_incl, axis=1)


def phrase_ndc(df):
    if(df['NDC_target'] == no_commitment_phrase):
        return f'{df["name"]} {no_commitment_phrase}'
    else:
        return f'{df["name"]} NDC target is {df["NDC_target"]} mtCO2e/yr by 2030. This country\'s {df["NDC_update"]} {df["NDC_include"]}'


ndc['value'] = ndc.apply(phrase_ndc, axis=1)
ndc.drop(columns=['NDC_target','NDC_update','NDC_include'], inplace=True)
ndc['indicator'] = 'nationally_determined_contributions'

ndc

,iso,name,value,indicator
0,AGO,Angola,Angola NDC target is 96.65 mtCO2e/yr by 2030. ...,nationally_determined_contributions
1,ATG,Antigua & Barbuda,Antigua & Barbuda has no NDC commitment,nationally_determined_contributions
2,AUS,Australia,Australia NDC target is 161.0 mtCO2e/yr by 203...,nationally_determined_contributions
3,BHR,Bahrain,Bahrain has no NDC commitment,nationally_determined_contributions
4,BGD,Bangladesh,Bangladesh NDC target is 35.1 mtCO2e/yr by 203...,nationally_determined_contributions
...,...,...,...,...
96,VUT,Vanuatu,Vanuatu NDC target is 0.07 mtCO2e/yr by 2030. ...,nationally_determined_contributions
97,VEN,Venezuela,Venezuela NDC target is 68.0 mtCO2e/yr by 2030...,nationally_determined_contributions
98,VNM,Vietnam,Vietnam NDC target is 196.85 mtCO2e/yr by 2030...,nationally_determined_contributions
99,VGB,"Virgin Islands, British","Virgin Islands, British has no NDC commitment",nationally_determined_contributions


In [5]:
country = 'Indonesia'
ndc[ndc['name'] == country]

,iso,name,value,indicator
40,IDN,Indonesia,Indonesia NDC target is 1181.21 mtCO2e/yr by 2...,nationally_determined_contributions


## Forest Reference Emission Levels (FRELs)

Sentence:  
`Indonesia` 2017 FREL is `48.9` Mt CO₂e/yr (Indonesia’s mangroves are considered `Forest/Wetland`  

Logic:
1. FREL value is provided
2. forest or wtland consideration is included in the **forest_wetland** field


In [6]:
country = 'Indonesia'
policy_data[policy_data['name'] == country]['frel']

40    593329235.0
Name: frel, dtype: float64

In [17]:
frel = policy_data[['iso','name']].copy()

def frel_sentence(df):
    if(df['frel'] > 0):
        if(df['forest_wetland'] == 'forest'):
            is_forest = f' (This country\'s mangroves are considered forest).'
        else:
            is_forest = ''
        
        return f'{df["name"]} 2017 FREL is {round(np.int64(df["frel"])/1000000, 2)} Mt CO₂e/yr.{is_forest}'
    else:
        return '-'

frel['value'] = policy_data.apply(frel_sentence, axis = 1)
frel['indicator'] = 'forest_reference_emissions_levels'
frel.head(15)

,iso,name,value,indicator
0,AGO,Angola,-,forest_reference_emissions_levels
1,ATG,Antigua & Barbuda,-,forest_reference_emissions_levels
2,AUS,Australia,-,forest_reference_emissions_levels
3,BHR,Bahrain,-,forest_reference_emissions_levels
4,BGD,Bangladesh,Bangladesh 2017 FREL is 0.37 Mt CO₂e/yr.,forest_reference_emissions_levels
5,BLZ,Belize,Belize 2017 FREL is 5.58 Mt CO₂e/yr.,forest_reference_emissions_levels
6,BEN,Benin,-,forest_reference_emissions_levels
7,BES,"Bonaire, Sint-Eustasius, Saba",-,forest_reference_emissions_levels
8,BRA,Brazil,Brazil 2017 FREL is 751.78 Mt CO₂e/yr. (This c...,forest_reference_emissions_levels
9,BRN,Brunei,-,forest_reference_emissions_levels


## Wetands supplement

Sentence:  
Indonesia `has/has not` implemented the Wetlands Supplement

Logic:  
The value is supplied in the field **wetland_suplement** 

In [8]:
wetland = policy_data[['iso','name']].copy()

def wetland_sentence(df):
    return f'{df["name"]} {df["wetland_suplement"].lower()} implemented the Wetlands Supplement'

wetland['indicator'] = 'ipcc_wetlands_supplement'
wetland['value'] = policy_data.apply(wetland_sentence, axis= 1)
wetland.head(15)


,iso,name,indicator,value
0,AGO,Angola,ipcc_wetlands_supplement,Angola has not implemented the Wetlands Supple...
1,ATG,Antigua & Barbuda,ipcc_wetlands_supplement,Antigua & Barbuda has not implemented the Wetl...
2,AUS,Australia,ipcc_wetlands_supplement,Australia has implemented the Wetlands Supplement
3,BHR,Bahrain,ipcc_wetlands_supplement,Bahrain has not implemented the Wetlands Suppl...
4,BGD,Bangladesh,ipcc_wetlands_supplement,Bangladesh has not implemented the Wetlands Su...
5,BLZ,Belize,ipcc_wetlands_supplement,Belize has not implemented the Wetlands Supple...
6,BEN,Benin,ipcc_wetlands_supplement,Benin has not implemented the Wetlands Supplement
7,BES,"Bonaire, Sint-Eustasius, Saba",ipcc_wetlands_supplement,"Bonaire, Sint-Eustasius, Saba has not implemen..."
8,BRA,Brazil,ipcc_wetlands_supplement,Brazil has not implemented the Wetlands Supple...
9,BRN,Brunei,ipcc_wetlands_supplement,Brunei has not implemented the Wetlands Supple...


## Combine and add location id

In [19]:
df_combined = pd.concat([ndc, frel, wetland])
df_combined

,iso,name,value,indicator
0,AGO,Angola,Angola NDC target is 96.65 mtCO2e/yr by 2030. ...,nationally_determined_contributions
1,ATG,Antigua & Barbuda,Antigua & Barbuda has no NDC commitment,nationally_determined_contributions
2,AUS,Australia,Australia NDC target is 161.0 mtCO2e/yr by 203...,nationally_determined_contributions
3,BHR,Bahrain,Bahrain has no NDC commitment,nationally_determined_contributions
4,BGD,Bangladesh,Bangladesh NDC target is 35.1 mtCO2e/yr by 203...,nationally_determined_contributions
...,...,...,...,...
96,VUT,Vanuatu,Vanuatu has not implemented the Wetlands Suppl...,ipcc_wetlands_supplement
97,VEN,Venezuela,Venezuela has not implemented the Wetlands Sup...,ipcc_wetlands_supplement
98,VNM,Vietnam,Vietnam has not implemented the Wetlands Suppl...,ipcc_wetlands_supplement
99,VGB,"Virgin Islands, British","Virgin Islands, British has not implemented th...",ipcc_wetlands_supplement


In [20]:
# Import locations to get staging ids
locations = pd.read_csv('../../../data/staging_locations.csv')
locations = locations[locations['location_type'] == 'country']
locations = locations[['id', 'name', 'iso']].copy()
locations.rename(columns={'id':'location_id'}, inplace=True)
locations.head()

,location_id,name,iso
159,1402,Dominican Republic,DOM
160,1401,Colombia,COL
161,1400,"Congo, DRC",COD
162,1399,Australia,AUS
163,1398,Angola,AGO


In [21]:
df_final = pd.merge(df_combined, locations[['location_id','iso']], on='iso', how='left')
df_final = df_final[~df_final['location_id'].isna()]
df_final = df_final[['location_id', 'indicator', 'value']]
df_final

,location_id,indicator,value
0,1398,nationally_determined_contributions,Angola NDC target is 96.65 mtCO2e/yr by 2030. ...
1,1370,nationally_determined_contributions,Antigua & Barbuda has no NDC commitment
2,1399,nationally_determined_contributions,Australia NDC target is 161.0 mtCO2e/yr by 203...
3,1374,nationally_determined_contributions,Bahrain has no NDC commitment
4,1373,nationally_determined_contributions,Bangladesh NDC target is 35.1 mtCO2e/yr by 203...
...,...,...,...
298,1394,ipcc_wetlands_supplement,Vanuatu has not implemented the Wetlands Suppl...
299,1362,ipcc_wetlands_supplement,Venezuela has not implemented the Wetlands Sup...
300,1364,ipcc_wetlands_supplement,Vietnam has not implemented the Wetlands Suppl...
301,1363,ipcc_wetlands_supplement,"Virgin Islands, British has not implemented th..."


In [22]:
df_final.to_csv('../../../data/international_status_widget_data.csv', index=False)